Grab the parameters needed from the merger trees


In [2]:
#copied from join time code
import iapi_TNG as iapi
import numpy as np
import h5py
import requests
from urllib.error import HTTPError
import pickle

#Box boundary 
L=75000. #kpc
halfbox=L/2.   
h=0.6774 

#Administrative URL pulling
baseUrl = 'http://www.tng-project.org/api/'
# headers = {"api-key":"env{apikey}"}

r=iapi.get(baseUrl)
names = [sim['name'] for sim in r['simulations']]
i = names.index('TNG100-1')
sim = iapi.get( r['simulations'][i]['url'] )
snaps = iapi.get( sim['snapshots'] )


In [3]:
from os import path
#also pulled from join time code
def getsub(snapnum,subid):
    #Pull url of a sub
    #subid is the ID back into the subhalo group catalog
    url= 'https://www.tng-project.org/api/TNG100-1/snapshots/'+str(snapnum)+'/subhalos/'+str(subid)+'/'
    sub=iapi.get(url)
    return(sub)

def gettree(subid):
    #pull the z=0 merger tree for a subhlao
    #some subhalos don't have merger trees
    fName = 'Trees/sublink_mpb_'+str(subid)
    if path.exists(fName+'.hdf5'):
        return(fName+'.hdf5')
    url='https://www.tng-project.org/api/TNG100-1/snapshots/99/subhalos/'+str(subid)+'/sublink/mpb.hdf5'
    tree=iapi.get(url,fName=fName)
    return(tree)

def getredshift(snapnum):
    #convert a snapshot number to a redshift
    return(snaps[snapnum]['redshift'])

In [4]:
#example test. snapshot 98 corresponds to z~0.0095
getredshift(98)

0.00952166696794476

In [5]:
#get current directory
currentdirc=!pwd
print(currentdirc)
dirc= ' '.join(currentdirc) + '/TNG_workshop_data/'
print(f"current dirc {dirc}")

['/Users/alexpoulin/Downloads/git/TNG/TNG_workshop']
current dirc /Users/alexpoulin/Downloads/git/TNG/TNG_workshop/TNG_workshop_data/


In [8]:
#pull my data 
#dat=np.load(dirc + '/projectnb/gravlens/bnmcd/SFR/catalogs/subhalocat.npy',allow_pickle=True).item()
dat=np.load(dirc + 'galcat.npy',allow_pickle=True).item()

#for requirement imposed after catalog had already been defined 
#N_stars=np.loadtxt('/projectnb/gravlens/bnmcd/SFR/catalogs/N_stars.txt')
N_stars = 5000

In [9]:
#pull the relevent fields from my data
subid = dat['subID'].astype('int')
satclass=dat['class']
sats = subid[satclass==1]
classN = satclass[N_stars>1000]
hostid = dat['subhostID'].astype('int')#[N_stars>1000]

print(len(classN[classN==1]))
#print(snaps[99].keys())

KeyError: 'class'

In [ ]:
#dat.keys()
#example converting stellar mass from code units to physical units
s_mass = np.log10(dat['s_mass'][N_stars>1000]*10**10/h)
#print(s_mass[0])
print(len(s_mass[s_mass>10.5]))
print(len(s_mass[s_mass<10]))

2167
2709


### One sub test

Pulling the merger tree for the most massive subhalo in the simulation

In [10]:
mpb1 = gettree(0)
f = h5py.File(mpb1,'r')
f.keys()

FileNotFoundError: [Errno 2] No such file or directory: 'Trees/sublink_mpb_0.hdf5'

In [ ]:
f['SnapNum'][:]
#note that the array is ordered such that the most recent snapshot is the first entry

array([99, 98, 97, 96, 95, 94, 93, 92, 91, 90, 89, 88, 87, 86, 85, 84, 83,
       82, 81, 80, 79, 78, 77, 76, 75, 74, 73, 72, 71, 70, 69, 68, 67, 66,
       65, 64, 63, 62, 61, 60, 59, 58, 57, 56, 55, 54, 53, 52, 51, 50, 49,
       48, 47, 46, 45, 44, 43, 42, 41, 40, 39, 38, 37, 36, 35, 34, 33, 32,
       31, 30, 29, 28, 27, 26, 25, 24, 23, 22, 21, 20, 19, 18, 17, 16, 15,
       14, 13, 12, 11, 10,  9,  8,  7,  6,  5,  4,  3,  2,  1],
      dtype=int16)

## Real Join Time
Note that this code defines joining time as the time since a satellite first approached its FoF group center within 3*R200 (R200 is a measurement of radius).

For magnetic field project, define joining time as the earliest time when the host and satellite had the same SubhaloGrNr 

In [ ]:
def jointime(ID,hostID):
    """
    Use the satellite and host trees to find the joining redshift of a satellite based on when it first approached its z=0 FoF group within 3R200
    Identify the change in satellite parameters since they joined
    """


    #fetch the merger tree for the satellite
    mpb1 = gettree(ID)
    f = h5py.File(mpb1,'r')
    #grPos = f['GroupPos'][:]
    subPos = f['SubhaloPos'][:]
    #grR200 = f['Group_R_Crit200'][:]
    snapnum= f['SnapNum'][:]
    #grVel = f['GroupVel'][:]
    subVel = f['SubhaloVel'][:]
    subMasstype = f['SubhaloMassType'][:]
    progID = f['SubfindID'][:]
    
    #fetch the merger tree for the satellite's host

    mpbhost =gettree(hostID)
    #print(mpbhost)
    fh = h5py.File(mpbhost,'r')
    grPos = fh['GroupPos'][:]
    grR200 = fh['Group_R_Crit200'][:]
    grVel = fh['SubhaloVel'][:]
    hostprog = fh['SubhaloID']
    

    # to compare distances, first make arrays the same shapes
    # sometimes trees are "truncated," meaning they don't go all the way back to the start of the simulation
    if len(grPos)>len(subPos):         
        grPos=grPos[0:len(subPos),:]
        grR200=grR200[0:len(subPos)]
        grVel = grVel[0:len(subPos),:]

    elif len(grPos)<len(subPos):
        subPos=subPos[0:len(grPos),:]
        subVel=subVel[0:len(grPos),:]
        snapnum = snapnum[0:len(grPos)]
        subMasstype = subMasstype[0:len(grPos),:]

    
    #Taking a square root is computationally inefficient.
    #Because distance in 3D is calculated with x^2+y^2+z^2 = distance^2,
    #I just work with distances in squares until the last step

    grR200sq = 9*np.multiply(grR200,grR200) #looking within 3R200 to get joining time for more satellites
    
    #Find the relative distance between two galaxies
    difpos=np.subtract(subPos,grPos)
    
    #The simulation uses repeating boundary conditions. 
    # So two galaxies can be quite close, even if they are technically at either end of the box
    #I can explain further when we meet to discuss

    #Replace values that are affected by boundary conditions
    difpos = np.where(abs(difpos)>halfbox,abs(difpos)-L, difpos)
    
    #Find the distance square between satellite and central as a function of redshift
    distsq=np.sum(np.square(difpos),axis=1)


    wh=np.nonzero((distsq<grR200sq))
    
    #print(wh.any)
    #Separation at z=0
    sep_z0 = np.sqrt(distsq[0])
    sep_norm = sep_z0/grR200[0]
    #Find when the 
    closeind = np.argmin(distsq)
    closest = np.sqrt(distsq[closeind])
    closest_norm = closest/grR200[closeind]
    closest_z = getredshift(snapnum[closeind])
    
    if len(wh[0])==0: 
        #in some cases, the satellite has never approached within the required distance
        #this shouldn't trigger when satellite joining redshift is defined by SubhaloGrNr
        print('not within 3')
        return(np.nan,sep_z0,sep_norm, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, closest,closest_norm,closest_z, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan)
    
    #find the index at which the satellite joined 
    joinind = max(wh[0])
    #print(snapnum[joinind])
    #print(grR200sq[wh],distsq[wh])
    whinside=snapnum[wh]
    whID = progID[wh] #ID of the "progenitor" galaxy of the subhalo, used to pull progenitor cutout in other code
    #print(whinside)
    minind = np.argmin(whinside)
    
    joinsnap=whinside[minind]
    joinprog = whID[minind]
    #print(joinsnap, snapnum, len(snapnum))
    joinred = getredshift(joinsnap)
    #joinind = 99-joinsnap
    #if joinind == len(snapnum): joinind=-1
    
    #Join scale factor (a), a=1 at z=0
    a=1./(1+joinred) #needed for converting comoving kiloparsecs, don't worry about this for magnetic field project
    
    #Get change in parameters since satellite joined:
    
    #Change in gas mass
    M_gas_0 = subMasstype[0][0]
    M_gas_join = subMasstype[joinind][0]
    del_M = M_gas_0 - M_gas_join
    
    #to understand kinetic energy, want to know the typical change in different masses
    del_M_total = np.sum(subMasstype[0]) - np.sum(subMasstype[joinind])
    del_M_dm = subMasstype[0][1] - subMasstype[joinind][1]
    del_M_stars = subMasstype[0][4] - subMasstype[joinind][4]
    
    #Change in kinetic energy (both all mass and star and gas mass)
    M_all_0 = np.sum(subMasstype[0])
    M_all_join = np.sum(subMasstype[joinind])
    M_lim_0 = M_gas_0 + subMasstype[0][4] #gas+stars
    M_lim_join = M_gas_join + subMasstype[joinind][4]
    
    rel_velsq_0 = np.sum(np.square((subVel[0]-grVel[0])))
    #print(rel_velsq_0)
    rel_velsq_join = np.sum(np.square((subVel[joinind]-grVel[joinind]*(1/a))))
    
    del_vsq = rel_velsq_0-rel_velsq_join
    
    #kinetic energy
    T_all_0 = 0.5*M_all_0*rel_velsq_0
    T_all_join = 0.5*M_all_join*rel_velsq_join
    del_T_all = T_all_0 - T_all_join
    
    T_lim_0 = 0.5*M_lim_0*rel_velsq_0
    T_lim_join = 0.5*M_lim_join*rel_velsq_join
    del_T_lim = T_lim_0 - T_lim_join
    
    #angular momentum
    L_0 = M_all_0*np.sqrt(rel_velsq_0)*sep_z0
    L_join = M_all_join*np.sqrt(rel_velsq_join)*np.sqrt(distsq[joinind])
    del_L = L_0-L_join
    
    print(joinsnap)
    
    s_mass_j = subMasstype[joinind][4]
    
    hostprog_ID  = hostprog[joinind]
    
    return(joinred,sep_z0,sep_norm, del_M, del_T_all, del_T_lim, del_M_total,del_M_dm, del_M_stars,del_vsq,joinsnap,closest, closest_norm, closest_z, joinprog, del_L, s_mass_j, hostprog_ID, L_join, L_0)


In [ ]:
#test for one satellite, central pair
jointime(subid[21],0)

Trees/sublink_mpb_0.hdf5
41


(1.41409822037252,
 278.81686,
 0.27025825,
 -19.652775,
 -106889906.54809189,
 -11480218.522767663,
 -156.1873,
 -138.49753,
 1.9622335,
 294808.0,
 41,
 249.60417,
 0.2508941,
 0.152748768902381,
 133812)

## iterate over all satellites

In [11]:
joinred = np.full(len(subid), np.nan)
sep = np.full(len(subid), np.nan)
sep_norm = np.full(len(subid), np.nan)
del_M = np.full(len(subid), np.nan)
del_T_all = np.full(len(subid), np.nan)
del_T_lim = np.full(len(subid), np.nan)
del_M_total = np.full(len(subid), np.nan)
del_M_dm = np.full(len(subid), np.nan)
del_M_stars = np.full(len(subid), np.nan)
del_vsq = np.full(len(subid), np.nan)
joinsnap = np.full(len(subid), np.nan)
closest = np.full(len(subid), np.nan)
closest_norm = np.full(len(subid), np.nan)
closest_z = np.full(len(subid), np.nan)
joinprog = np.full(len(subid), np.nan)
del_L  = np.full(len(subid), np.nan)
s_mass_j = np.full(len(subid),np.nan)
hostprogID = np.full(len(subid), np.nan)
L_join = np.full(len(subid), np.nan)
L_0  = np.full(len(subid),np.nan)

for i in range(0,len(subid)):
    if satclass[i] ==0: continue
    if N_stars[i]<1000: continue
    
    #print(subid[i],hostid[i])
    joinred[i], sep[i], sep_norm[i], del_M[i], del_T_all[i], del_T_lim[i], del_M_total[i],del_M_dm[i],del_M_stars[i], del_vsq[i], joinsnap[i], closest[i],closest_norm[i], closest_z[i], joinprog[i], del_L[i], s_mass_j[i], hostprogID[i], L_join[i], L_0[i] = jointime(subid[i],hostid[i] )
    

94
82
96
98
79
78
55
80
61
68
49
58
53
80
98
51
96
61
43
92
43
90
54
65
67
65
85
65
96
98
94
83
74
95
68
56
34
57
53
91
63
93
96
45
43
98
78
91
57
75
58
78
38
80
54
46
90
39
79
74
82
81
74
74
78
55
79
73
75
73
77
74
44
71
76
45
72
86
77
72
84
70
48
80
59
78
82
79
46
59
67
71
79
81
81
81
45
79
76
73
45
72
56
82
61
76
65
68
59
52
76
41
35
56
66
84
71
62
48
43
64
42
38
77
37
75
53
51
53
55
73
48
47
78
55
74
55
42
55
52
46
73
55
89
not within 4
81
93
83
84
not within 4
not within 4
63
91
71
61
78
99
not within 4
46
92
not within 4
65
not within 4
53
92
96
78
not within 4
97
68
96
43
51
77
38
94
39
52
49
72
83
59
85
71
59
86
70
85
84
82
61
84
60
60
49
70
62
65
61
61
40
53
72
43
56
45
88
92
60
88
72
53
94
72
89
76
82
44
86
82
65
58
53
73
84
47
77
56
74
50
77
45
58
82
67
73
47
43
45
79
55
54
51
63
55
66
55
85
55
53
70
74
40
70
47
46
55
41
83
52
44
37
46
37
90
not within 4
70
not within 4
not within 4
not within 4
81
42
65
79
not within 4
85
51
74
not within 4
79
40
50
not within 4
58
33
42
no

Trees/sublink_mpb_114388.hdf5
77
114396 114388
Trees/sublink_mpb_114388.hdf5
86
114398 114388
Trees/sublink_mpb_114388.hdf5
48
114401 114388
Trees/sublink_mpb_114388.hdf5
77
114409 114388
Trees/sublink_mpb_114388.hdf5
77
114425 114388
Trees/sublink_mpb_114388.hdf5
77
118681 118679
Trees/sublink_mpb_118679.hdf5
53
118686 118679
Trees/sublink_mpb_118679.hdf5
65
118687 118679
Trees/sublink_mpb_118679.hdf5
55
118701 118679
Trees/sublink_mpb_118679.hdf5
39
118704 118679
Trees/sublink_mpb_118679.hdf5
36
118713 118679
Trees/sublink_mpb_118679.hdf5
49
118715 118679
Trees/sublink_mpb_118679.hdf5
36
118720 118679
Trees/sublink_mpb_118679.hdf5
42
118729 118679
Trees/sublink_mpb_118679.hdf5
66
121864 121863
Trees/sublink_mpb_121863.hdf5
70
121865 121863
Trees/sublink_mpb_121863.hdf5
68
121866 121863
Trees/sublink_mpb_121863.hdf5
61
121871 121863
Trees/sublink_mpb_121863.hdf5
74
121874 121863
Trees/sublink_mpb_121863.hdf5
47
121876 121863
Trees/sublink_mpb_121863.hdf5
41
121878 121863
Trees/sublink

Trees/sublink_mpb_168390.hdf5
75
168402 168390
Trees/sublink_mpb_168390.hdf5
70
168415 168390
Trees/sublink_mpb_168390.hdf5
66
168417 168390
Trees/sublink_mpb_168390.hdf5
67
168421 168390
Trees/sublink_mpb_168390.hdf5
45
170541 170540
Trees/sublink_mpb_170540.hdf5
73
170542 170540
Trees/sublink_mpb_170540.hdf5
77
170543 170540
Trees/sublink_mpb_170540.hdf5
60
170545 170540
Trees/sublink_mpb_170540.hdf5
42
170569 170540
Trees/sublink_mpb_170540.hdf5
41
172650 172649
Trees/sublink_mpb_172649.hdf5
not within 4
172651 172649
Trees/sublink_mpb_172649.hdf5
88
172652 172649
Trees/sublink_mpb_172649.hdf5
95
172653 172649
Trees/sublink_mpb_172649.hdf5
92
172654 172649
Trees/sublink_mpb_172649.hdf5
not within 4
172655 172649
Trees/sublink_mpb_172649.hdf5
94
172658 172649
Trees/sublink_mpb_172649.hdf5
57
172660 172649
Trees/sublink_mpb_172649.hdf5
not within 4
172661 172649
Trees/sublink_mpb_172649.hdf5
not within 4
172669 172649
Trees/sublink_mpb_172649.hdf5
36
175239 175238
Trees/sublink_mpb_17

Trees/sublink_mpb_223896.hdf5
79
223900 223896
Trees/sublink_mpb_223896.hdf5
77
225156 225155
Trees/sublink_mpb_225155.hdf5
85
225158 225155
Trees/sublink_mpb_225155.hdf5
84
225159 225155
Trees/sublink_mpb_225155.hdf5
45
225160 225155
Trees/sublink_mpb_225155.hdf5
90
226196 226194
Trees/sublink_mpb_226194.hdf5
82
226197 226194
Trees/sublink_mpb_226194.hdf5
53
226199 226194
Trees/sublink_mpb_226194.hdf5
75
226220 226194
Trees/sublink_mpb_226194.hdf5
37
227576 227574
Trees/sublink_mpb_227574.hdf5
70
227583 227574
Trees/sublink_mpb_227574.hdf5
85
228681 228680
Trees/sublink_mpb_228680.hdf5
79
228682 228680
Trees/sublink_mpb_228680.hdf5
72
228683 228680
Trees/sublink_mpb_228680.hdf5
88
228684 228680
Trees/sublink_mpb_228680.hdf5
47
230236 230235
Trees/sublink_mpb_230235.hdf5
63
230237 230235
Trees/sublink_mpb_230235.hdf5
79
230238 230235
Trees/sublink_mpb_230235.hdf5
72
231707 231706
Trees/sublink_mpb_231706.hdf5
92
231708 231706
Trees/sublink_mpb_231706.hdf5
78
231709 231706
Trees/sublink

79
276221 276219
Trees/sublink_mpb_276219.hdf5
75
276222 276219
Trees/sublink_mpb_276219.hdf5
59
276223 276219
Trees/sublink_mpb_276219.hdf5
72
276236 276219
Trees/sublink_mpb_276219.hdf5
57
278303 278302
Trees/sublink_mpb_278302.hdf5
71
278304 278302
Trees/sublink_mpb_278302.hdf5
72
278855 278854
Trees/sublink_mpb_278854.hdf5
59
279708 279707
Trees/sublink_mpb_279707.hdf5
59
279709 279707
Trees/sublink_mpb_279707.hdf5
71
281701 281700
Trees/sublink_mpb_281700.hdf5
90
281702 281700
Trees/sublink_mpb_281700.hdf5
82
281705 281700
Trees/sublink_mpb_281700.hdf5
81
281706 281700
Trees/sublink_mpb_281700.hdf5
69
281707 281700
Trees/sublink_mpb_281700.hdf5
56
282378 282377
Trees/sublink_mpb_282377.hdf5
66
282379 282377
Trees/sublink_mpb_282377.hdf5
78
282381 282377
Trees/sublink_mpb_282377.hdf5
77
284317 284316
Trees/sublink_mpb_284316.hdf5
76
284318 284316
Trees/sublink_mpb_284316.hdf5
64
284949 284948
Trees/sublink_mpb_284948.hdf5
64
284950 284948
Trees/sublink_mpb_284948.hdf5
53
284951 284

Trees/sublink_mpb_331293.hdf5
57
331702 331701
Trees/sublink_mpb_331701.hdf5
61
332069 332068
Trees/sublink_mpb_332068.hdf5
72
332070 332068
Trees/sublink_mpb_332068.hdf5
78
332391 332389
Trees/sublink_mpb_332389.hdf5
55
332393 332389
Trees/sublink_mpb_332389.hdf5
63
332814 332812
Trees/sublink_mpb_332812.hdf5
57
333259 333258
Trees/sublink_mpb_333258.hdf5
64
333260 333258
Trees/sublink_mpb_333258.hdf5
81
333673 333672
Trees/sublink_mpb_333672.hdf5
85
333674 333672
Trees/sublink_mpb_333672.hdf5
84
333675 333672
Trees/sublink_mpb_333672.hdf5
87
334031 334028
Trees/sublink_mpb_334028.hdf5
88
334728 334727
Trees/sublink_mpb_334727.hdf5
54
335487 335486
Trees/sublink_mpb_335486.hdf5
76
336518 336516
Trees/sublink_mpb_336516.hdf5
70
337049 337048
Trees/sublink_mpb_337048.hdf5
66
337051 337048
Trees/sublink_mpb_337048.hdf5
74
337445 337444
Trees/sublink_mpb_337444.hdf5
56
337844 337843
Trees/sublink_mpb_337843.hdf5
84
337846 337843
Trees/sublink_mpb_337843.hdf5
87
337848 337843
Trees/sublink

Trees/sublink_mpb_392368.hdf5
90
392370 392368
Trees/sublink_mpb_392368.hdf5
81
392587 392585
Trees/sublink_mpb_392585.hdf5
80
392853 392852
Trees/sublink_mpb_392852.hdf5
73
393118 393117
Trees/sublink_mpb_393117.hdf5
84
393337 393336
Trees/sublink_mpb_393336.hdf5
78
393338 393336
Trees/sublink_mpb_393336.hdf5
73
393548 393547
Trees/sublink_mpb_393547.hdf5
44
394242 394241
Trees/sublink_mpb_394241.hdf5
94
394440 394439
Trees/sublink_mpb_394439.hdf5
73
394987 394986
Trees/sublink_mpb_394986.hdf5
55
395154 395153
Trees/sublink_mpb_395153.hdf5
64
395155 395153
Trees/sublink_mpb_395153.hdf5
71
395534 395533
Trees/sublink_mpb_395533.hdf5
72
395736 395735
Trees/sublink_mpb_395735.hdf5
72
395883 395882
Trees/sublink_mpb_395882.hdf5
66
396074 396073
Trees/sublink_mpb_396073.hdf5
81
396075 396073
Trees/sublink_mpb_396073.hdf5
66
396267 396266
Trees/sublink_mpb_396266.hdf5
73
396463 396462
Trees/sublink_mpb_396462.hdf5
87
397569 397568
Trees/sublink_mpb_397568.hdf5
95
397683 397682
Trees/sublink

Trees/sublink_mpb_459243.hdf5
34
459396 459394
Trees/sublink_mpb_459394.hdf5
71
460527 460526
Trees/sublink_mpb_460526.hdf5
63
461203 461202
Trees/sublink_mpb_461202.hdf5
59
461284 461283
Trees/sublink_mpb_461283.hdf5
81
462242 462241
Trees/sublink_mpb_462241.hdf5
62
462905 462904
Trees/sublink_mpb_462904.hdf5
81
462987 462986
Trees/sublink_mpb_462986.hdf5
not within 4
462988 462986
Trees/sublink_mpb_462986.hdf5
77
463140 463139
Trees/sublink_mpb_463139.hdf5
88
463522 463521
Trees/sublink_mpb_463521.hdf5
79
465362 465361
Trees/sublink_mpb_465361.hdf5
68
465414 465412
Trees/sublink_mpb_465412.hdf5
64
466266 466265
Trees/sublink_mpb_466265.hdf5
69
466600 466599
Trees/sublink_mpb_466599.hdf5
76
466695 466694
Trees/sublink_mpb_466694.hdf5
53
467446 467445
Trees/sublink_mpb_467445.hdf5
51
467629 467628
Trees/sublink_mpb_467628.hdf5
78
467741 467740
Trees/sublink_mpb_467740.hdf5
48
467872 467871
Trees/sublink_mpb_467871.hdf5
77
468065 468064
Trees/sublink_mpb_468064.hdf5
46
468625 468624
Tre

In [ ]:
#save satellite parameters to auxcat

auxcat=np.load('paper3/auxcat.npy',allow_pickle=True).item()
auxcat['joinred']=joinred
auxcat['sep'] = sep
auxcat['sep_norm'] = sep_norm
auxcat['del_M'] = del_M
auxcat['del_T_all'] = del_T_all
auxcat['del_T_lim'] = del_T_lim
auxcat['del_M_total'] = del_M_total
auxcat['del_M_dm'] = del_M_dm
auxcat['del_M_stars'] = del_M_stars
auxcat['del_vsq'] = del_vsq
auxcat['joinsnap'] = joinsnap
auxcat['closest'] = closest
auxcat['closest_norm'] = closest_norm
auxcat['closest_z'] = closest_z
auxcat['joinprog'] = joinprog
auxcat['del_L'] = del_L
auxcat['s_mass_j'] = s_mass_j
auxcat['hostprogID'] = hostprogID
auxcat['L_join']=L_join
auxcat['L_0'] = L_0
np.save('paper3/auxcat.npy',auxcat)